<a href="https://colab.research.google.com/github/stfc/janus-tutorials/blob/main/eos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equation of State with MLIPs

## Setup environment (Optional)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python3 -m pip install janus-core
!python3 -m pip install matgl
!python3 -m pip install chgnet
!python3 -m pip uninstall numpy -y
!python3 -m pip install numpy==1.25.2 # Colab compatibility

## Equation of State

In [ ]:
from ase import Atoms
from ase.io import read
from ase.visualize import view

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.eos import calc_eos

### Equation of state for α-quartz

Build the structure:

In [ ]:
α_quartz = Atoms(symbols=(*["Si"]*3, *["O"]*6),
                     scaled_positions=[[0.469700, 0.000000, 0.000000],
                                       [0.000000, 0.469700, 0.666667],
                                       [0.530300, 0.530300, 0.333333],

                                       [0.413500, 0.266900, 0.119100],
                                       [0.266900, 0.413500, 0.547567],
                                       [0.733100, 0.146600, 0.785767],

                                       [0.586500, 0.853400, 0.214233],
                                       [0.853400, 0.586500, 0.452433],
                                       [0.146600, 0.733100, 0.880900]],

                     cell=[[4.916000, 0.000000, 0.000000],
                           [-2.45800, 4.257381, 0.000000],
                           [0.000000, 0.000000, 5.405400]],
                     pbc=True,
                     )

view(α_quartz)

Calculate the equation of state using the MACE-MP potential:

In [ ]:
sp = SinglePoint(struct=α_quartz,
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small','default_dtype':'float64'})

mace_eos = calc_eos(
    sp.struct,
    minimize_kwargs={"filter_func": None},
    min_volume=0.75,
    max_volume=1.25,
    n_volumes=20,
)

In [ ]:
mace_eos["eos"].plot(show=True)

### Equation of state for β-quartz

Perform the same calculation for β-quartz:

In [ ]:
β_quartz = read("data/beta_quartz.cif")
view(β_quartz)

In [ ]:
sp_beta = SinglePoint(struct=β_quartz,
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small','default_dtype':'float64'})

mace_eos_beta = calc_eos(
    sp_beta.struct,
    minimize_kwargs={"filter_func": None},
    min_volume=0.75,
    max_volume=1.25,
    n_volumes=20,
)

In [ ]:
mace_eos_beta["eos"].plot(show=True)

Combining plots for α-quartz and β-quartz:

In [ ]:
import matplotlib.pyplot as plt

ax = plt.gca()

data_alpha = mace_eos["eos"].getplotdata()
data_beta = mace_eos_beta["eos"].getplotdata()

ax.plot(data_alpha[4], data_alpha[5], ls='-', color='C3', label="α-quartz")
ax.plot(data_alpha[6], data_alpha[7], ls='', marker='x', color='C4', mfc='C4')

ax.plot(data_beta[4], data_beta[5], ls='-', color='C0', label="β-quartz")
ax.plot(data_beta[6], data_beta[7], ls='', marker='x', color='C2', mfc='C2')

ax.set_xlabel('volume [Å$^3$]')
ax.set_ylabel('energy [eV]')
ax.legend()

plt.show()

### Comparing MACE to CHGNET and MGNET

In [ ]:
sp = SinglePoint(struct=α_quartz,
                 architecture="m3gnet",
                 device="cpu")

m3gnet_eos = calc_eos(sp.struct, minimize_kwargs={"filter_func": None})
m3gnet_eos["eos"].plot(show=True)

In [ ]:
sp = SinglePoint(struct=α_quartz,
                 architecture="chgnet",
                 device="cpu")

chgnet_eos = calc_eos(sp.struct, minimize_kwargs={"filter_func": None})
chgnet_eos["eos"].plot(show=True)

In [ ]:
print(f"MACE energy [eV]: {mace_eos['e_0']}")
print(f"M3GNET energy [eV]: {m3gnet_eos['e_0']}")
print(f"CHGNET energy [eV]: {chgnet_eos['e_0']}")

print()

print(f"MACE volume [Å^3]: {mace_eos['v_0']}")
print(f"M3GNET volume [Å^3]: {m3gnet_eos['v_0']}")
print(f"CHGNET volume [Å^3]: {chgnet_eos['v_0']}")

print()

print(f"MACE bulk_modulus [GPa]: {mace_eos['bulk_modulus']}")
print(f"M3GNET bulk_modulus [GPa]: {m3gnet_eos['bulk_modulus']}")
print(f"CHGNET bulk_modulus [GPa]: {chgnet_eos['bulk_modulus']}")